<a href="https://colab.research.google.com/github/aelricc/lmmm-notebook/blob/main/LMMM_Generic_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install Dependencies
You must run this first! (You can ignore the warnings, just a result of automatically restarting the runtime)

In [ ]:
!pip install --upgrade git+https://github.com/google/lightweight_mmm.git
!pip install -q xlrd
!pip install fpdf2 matplotlib
import os
os.kill(os.getpid(), 9)

  Cloning https://github.com/google/lightweight_mmm.git to /tmp/pip-req-build-rwxmf49v
  Running command git clone --filter=blob:none --quiet https://github.com/google/lightweight_mmm.git /tmp/pip-req-build-rwxmf49v
  Resolved https://github.com/google/lightweight_mmm.git to commit b4c99fa7532b767b7c3b519e41f8dffa15ab8c65
  Preparing metadata (setup.py) ... done


### Input Data
Make sure the data you want to input is

1.   An .xlsx file
2.   Stored in a folder called "LMMM-Data" on your Drive

Don't forget to change the SEED!



In [ ]:
import numpyro
#START HERE
SEED = 100 #@param {type:"raw"}


#You must have a folder called "LMMM-Data"
file_name = "data-simm5.xlsx" #@param {type:"string"}
sheet_name = 'Weekly'         #@param {type:"string"}

#specify location of data
media_data_names = "impressions_YouTube impressions_Instagram clicks_Search" #@param {type:"string"}
spend_names = "spend_YouTube spend_Instagram spend_Search" #@param {type:"string"}
target_names = "total_revenue" #@param {type:"string"}

#extra features, holiday details, geos?
other_feature_names = "" #@param {type:"string"}

#specify time range (leave blank for default, aka all dates)
start_date = "" #@param {type:"string"}
end_date = ""   #@param {type:"string"}


#How much of the data do you want to test?
test_size = 20 #@param {type:"integer"}

#CHANGE THE SEED IF YOU'RE DOING A NEW TEST
number_warmup=1000
number_samples=1000
degrees_seasonality=6

weekly = True #@param {type: "boolean"}

model_name = "hill_adstock" #@param ["hill_adstock", "carryover", "adstock"]

default_priors = True #@param {type:"boolean"}

if default_priors:
  baseline_sales_contribution = 2

  #hill-adstock
  saturation_point = 1.
  halfway_marginal_gains = 1.

  #carryover
  retention_rate_distribution = 1.
  peak_effect_point = 1.
else:
  baseline_sales_contribution = 2 #@param {type:"raw"}

  #hill-adstock
  saturation_point = 1. #@param {type:"raw"}
  halfway_marginal_gains = 1. #@param {type:"raw"}

  #carryover
  retention_rate_distribution = 1.  #@param {type:"raw"}
  peak_effect_point = 1. #@param {type:"raw"}



intercept = numpyro.distributions.HalfNormal(scale=baseline_sales_contribution)

#hill-adstock
K_rate = 1.
half_max_effective_concentration = numpyro.distributions.Gamma(concentration=saturation_point, rate=K_rate)
slope_rate = 1.
slope = numpyro.distributions.Gamma(concentration=halfway_marginal_gains , rate=slope_rate)

#carryover
retentionrate_concentration_2 = 1.
ad_effect_retention_rate = numpyro.distributions.Beta(concentration1=retention_rate_distribution, concentration0=retentionrate_concentration_2)
peak_effect_delay = numpyro.distributions.HalfNormal(scale=peak_effect_point)

#adstock

lag_weight_concentration = 2  #@param {type:"raw"}
lagweight_concentration_2 = 1. #@param {type:"raw"}

lag_weight = numpyro.distributions.Beta(concentration1=lag_weight_concentration, concentration0=lagweight_concentration_2)



### Run the Model

Output will be in a new folder in "LMMM-Data"

(Here's mine) https://drive.google.com/drive/folders/11ciF7F-ut8rIhgwSgDNNp0UpY-jNIofT?usp=sharing

In [ ]:
from pandas.core.internals.managers import ensure_wrapped_if_datetimelike
#Importing data
from lightweight_mmm import preprocessing, lightweight_mmm, plot, optimize_media
import jax.numpy as jnp
import numpyro
from sklearn.metrics import mean_absolute_percentage_error
import pandas as pd
import os
import sys
from google.colab import drive

drive.mount('/content/drive')
path_to_file = '/content/drive/My Drive/LMMM-Data/' + file_name

data = pd.read_excel(path_to_file, sheet_name) #SELECT DESIRED SHEET
data_size = len(data)


print("Original Data Set: ")
print(data)


media_data_cols = media_data_names.split()
spend_cols = spend_names.split()
target_cols = target_names.split()
if other_feature_names: extra_feature_cols = other_feature_names.split()

#divide dataframe into media data, target, and cost for use in model

media_data = data.loc[:, media_data_cols]
print(media_data_cols)
print(media_data)

target = data.loc[:,target_cols]
costBS = data.loc[:, spend_cols]
if other_feature_names: extra_features = data.loc[:, extra_feature_cols]


if start_date and end_date:
  media_data = media_data.loc[start_date:end_date,...]
  target = target.loc[start_date:end_date,...]
  costBS = costBS.loc[start_date:end_date,...]
  if other_feature_names: extra_features = extra_features.loc[start_date:end_date,...]
elif start_date and not end_date:
  media_data = media_data.loc[start_date:,...]
  target = target.loc[start_date:,...]
  costBS = costBS.loc[start_date:,...]
  if other_feature_names: extra_features = extra_features.loc[start_date:,...]
elif not start_date and end_date:
  media_data = media_data.loc[:end_date,...]
  target = target.loc[:end_date,...]
  costBS = costBS.loc[:end_date,...]
  if other_feature_names: extra_features = extra_features.loc[:end_date,...]


print(media_data)

media_names = list(media_data.columns)
#media_data.shape

target = target.to_numpy().flatten()
#target.shape

unscaled_cost = costBS.sum().to_numpy()


print("\nMedia Data: ")
print(media_data)

print("\nTarget Data: ")
print(target)

print("\nSpend Data: ")
print(unscaled_cost)


#split data into training and testing data

split_point = data_size - test_size

media_data_train = (media_data.loc[:split_point, ...]).to_numpy()

media_data_test = (media_data.loc[split_point:, ...]).to_numpy()

target_train = (target[:(split_point+1)])
target_test = target[(split_point):]

if other_feature_names:
  extra_features_train = (extra_features.loc[:split_point, ...]).to_numpy()
  extra_features_test = (extra_features.loc[:split_point, ...]).to_numpy()


#print(target_test)

#scale the data

media_scaler = preprocessing.CustomScaler(divide_operation=jnp.mean)
target_scaler = preprocessing.CustomScaler(divide_operation=jnp.mean)
cost_scaler = preprocessing.CustomScaler(divide_operation=jnp.mean, multiply_by=1)

media_data_train = media_scaler.fit_transform(media_data_train)
target_train = target_scaler.fit_transform(target_train)
costs = cost_scaler.fit_transform(unscaled_cost)
if other_feature_names: extra_features_train = target_scaler.fit_transform(extra_features_train)

print(costs)

'''
print(media_data_train)
print(target_train)
'''

#run this to make sure data has been preprocessed appropiately
correlations, variances, spend_fractions, variance_inflation_factors = preprocessing.check_data_quality(
    media_data=media_scaler.transform(media_data),
    target_data=target_scaler.transform(target),
    cost_data=costs)
correlations[0].style.background_gradient(cmap='RdBu', vmin=-1, vmax=1).set_precision(3)
def highlight_variances(x: float,
                        low_variance_threshold: float=1.0e-3,
                        high_variance_threshold: float=3.0) -> str:

    if x < low_variance_threshold or x > high_variance_threshold:
      weight = 'bold'
      color = 'red'
    else:
      weight = 'normal'
      color = 'black'
    style = f'font-weight: {weight}; color: {color}'
    return style

variances.style.set_precision(4).applymap(highlight_variances)
def highlight_low_spend_fractions(x: float,
                                  low_spend_threshold: float=0.01) -> str:
    if x < low_spend_threshold:
      weight = 'bold'
      color = 'red'
    else:
      weight = 'normal'
      color = 'black'
    style = f'font-weight: {weight}; color: {color}'
    return style

spend_fractions.style.set_precision(4).applymap(highlight_low_spend_fractions)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Original Data Set: 
          DATE  impressions_YouTube  impressions_Instagram  spend_YouTube  \
0   2019-01-01         1.413640e+08           1.478478e+07  424251.295187   
1   2019-01-08         1.574656e+08           1.596988e+07  473207.183662   
2   2019-01-15         1.317644e+08           1.606780e+07  395621.056817   
3   2019-01-22         1.303690e+08           1.324150e+07  390489.494772   
4   2019-01-29         1.473974e+08           1.658058e+07  441941.250186   
..         ...                  ...                    ...            ...   
99  2020-11-24         1.262730e+08           1.380767e+07  378768.830841   
100 2020-12-01         1.308056e+08           1.431418e+07  392754.639475   
101 2020-12-08         1.179249e+08           1.302560e+07  353510.212512   
102 2020-12-15         1.333239e+08           1.552973e+07  400570.325299   
103 

<ipython-input-37-704d06566037>:118: FutureWarning: this method is deprecated in favour of `Styler.format(precision=..)`
  correlations[0].style.background_gradient(cmap='RdBu', vmin=-1, vmax=1).set_precision(3)
<ipython-input-37-704d06566037>:132: FutureWarning: this method is deprecated in favour of `Styler.format(precision=..)`
  variances.style.set_precision(4).applymap(highlight_variances)
<ipython-input-37-704d06566037>:144: FutureWarning: this method is deprecated in favour of `Styler.format(precision=..)`
  spend_fractions.style.set_precision(4).applymap(highlight_low_spend_fractions)


,fraction of spend
feature_0,0.1740
feature_1,0.1246
feature_2,0.7014


In [ ]:
parameterdict = {}

if model_name == "hill_adstock":
  custom_priors= {
      "intercept": intercept,
      "half_max_effective_concentration": half_max_effective_concentration,
      "slope": slope
      }
  for i in ('baseline_sales_contribution', 'saturation_point', 'halfway_marginal_gains'):
      parameterdict[i] = locals()[i]

elif model_name == "carryover":
  custom_priors= {
    "intercept": intercept,
    "ad_effect_retention_rate": ad_effect_retention_rate,
    "peak_effect_delay": peak_effect_delay
    }
  for i in ('baseline_sales_contribution', 'retention_rate_distribution', 'peak_effect_point'):
      parameterdict[i] = locals()[i]

elif model_name == "adstock":
  custom_priors= {
    "intercept": intercept,
    "lag_weight": lag_weight,
    }
  for i in ('baseline_sales_contribution', 'lag_weight_concentration', 'lagweight_concentration_2'):
    parameterdict[i] = locals()[i]

if weekly is True:
  weekday_seasonality=False
  seasonality_frequency=52
else:
  weekday_seasonality=True
  seasonality_frequency=365

media_prior = costs


mmm = lightweight_mmm.LightweightMMM(model_name)

if other_feature_names:
  mmm.fit(
      media=media_data_train,
      media_names=media_names,
      media_prior=media_prior,
      target=target_train,
      extra_features=extra_features_train,
      number_warmup=number_warmup,
      number_samples=number_samples,
      degrees_seasonality=degrees_seasonality,
      weekday_seasonality=weekday_seasonality,
      seasonality_frequency=seasonality_frequency,
      custom_priors = custom_priors,
      seed = SEED)
else:
  mmm.fit(
      media=media_data_train,
      media_names=media_names,
      media_prior=media_prior,
      target=target_train,
      number_warmup=number_warmup,
      number_samples=number_samples,
      degrees_seasonality=degrees_seasonality,
      weekday_seasonality=weekday_seasonality,
      seasonality_frequency=seasonality_frequency,
      custom_priors = custom_priors,
      seed = SEED)

/usr/local/lib/python3.10/dist-packages/lightweight_mmm/lightweight_mmm.py:358: UserWarning: There are not enough devices to run parallel chains: expected 2 but got 1. Chains will be drawn sequentially. If you are running MCMC in CPU, consider using `numpyro.set_host_device_count(2)` at the beginning of your program. You can double-check how many devices are available in your system using `jax.local_device_count()`.
  mcmc = numpyro.infer.MCMC(
sample: 100%|██████████| 2000/2000 [01:07<00:00, 29.46it/s, 1023 steps of size 2.75e-03. acc. prob=0.94]


In [ ]:

import lightweight_mmm.utils as utils
from matplotlib.backends.backend_pdf import PdfPages

from fpdf import FPDF
import matplotlib
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure
import seaborn as sns
from typing import Any, List, Optional, Sequence, Tuple
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt


plt1 = plot.plot_model_fit(mmm, target_scaler=target_scaler)

directory = file_name + '-' + str(SEED) + '-result'
parent_dir = "/content/drive/My Drive/LMMM-Data/"

path = os.path.join(parent_dir, directory)

if not os.path.exists(path):
  os.mkdir(path)

plt1.savefig(path + '/model_fit.pdf')

new_predictions = mmm.predict(media=media_scaler.transform(media_data_test),
                              seed=SEED)

plt2 = plot.plot_out_of_sample_model_fit(out_of_sample_predictions=new_predictions,
                                 out_of_sample_target=target_scaler.transform(target[split_point:]))
plt2.savefig(path + '/OOS_model_fit.pdf')

plt3 = plot.plot_media_channel_posteriors(media_mix_model=mmm, channel_names = media_names)
plt3.savefig(path + '/media_channel_posteriors.pdf')

#plt4 = plot.plot_prior_and_posterior(media_mix_model=mmm)
#plt4.savefig(path + '/prior_and_posterior.pdf')

media_contribution, roi_hat = mmm.get_posterior_metrics(target_scaler=target_scaler, cost_scaler=cost_scaler)

plt5 = plot.plot_media_baseline_contribution_area_plot(media_mix_model=mmm,
                                                target_scaler=target_scaler,
                                                channel_names = media_names,
                                                fig_size=(30,10))
plt5.savefig(path + '/media_baseline_contribution_area.pdf')


/usr/local/lib/python3.10/dist-packages/arviz/stats/stats_utils.py:509: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "stats_variance_1d" failed type inference due to: non-precise type pyobject
During: typing of argument at /usr/local/lib/python3.10/dist-packages/arviz/stats/stats_utils.py (511)

File "../usr/local/lib/python3.10/dist-packages/arviz/stats/stats_utils.py", line 511:
def stats_variance_1d(data, ddof=0):
    a_a, b_b = 0, 0
    ^

  @conditional_jit
/usr/local/lib/python3.10/dist-packages/arviz/stats/stats_utils.py:509: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "stats_variance_1d" failed type inference due to: Cannot determine Numba type of <class 'numba.core.dispatcher.LiftedLoop'>

File "../usr/local/lib/python3.10/dist-packages/arviz/stats/stats_utils.py", line 512:
def stats_variance_1d(data, ddof=0):
    <source elided>
    a_a, b_b = 0, 0
    for i in d

In [ ]:
def plot_bars_media_metrics_fixed(
    metric: jnp.ndarray,
    metric_name: str = "metric",
    channel_names: Optional[Tuple[Any]] = None,
    interval_mid_range: float = .9) -> matplotlib.figure.Figure:

  if channel_names is None:
    channel_names = np.arange(np.shape(metric)[1])
  upper_quantile = 1 - (1 - interval_mid_range) / 2
  lower_quantile = (1 - interval_mid_range) / 2

  if metric.ndim == 3:
    metric = jnp.mean(metric, axis=-1)

  if (metric_name == "Media Contribution Percentage"):
    metric = metric*(100)


  fig, ax = plt.subplots(1, 1)
  sns.barplot(data=metric, ci=None, ax=ax, orient='h')
  quantile_bounds = np.quantile(
      metric, q=[lower_quantile, upper_quantile], axis=0)
  quantile_bounds[0] = metric.mean(axis=0) - quantile_bounds[0]
  quantile_bounds[1] = quantile_bounds[1] - metric.mean(axis=0)

  if (metric_name == "Media Contribution Percentage"):
      ax.bar_label(ax.containers[0], fmt='%.2f%%')
  else:
      ax.bar_label(ax.containers[0])
  ax.set_yticks(range(len(channel_names)))
  ax.set_yticklabels(channel_names, rotation=0)
  fig.suptitle(
      f"Estimated media channel {metric_name}."
  )
  plt.tight_layout()
  plt.close()
  return fig


plt6 = plot_bars_media_metrics_fixed(metric=media_contribution, channel_names = media_names, metric_name="Media Contribution Percentage")
#plt6.set_figheight(10)
plt6.savefig(path + '/media_contribution_percentage.pdf')

plt10 = plot_bars_media_metrics_fixed(metric=costs, channel_names = media_names, metric_name="Spend Data")


plt7 = plot_bars_media_metrics_fixed(metric=roi_hat, channel_names = media_names, metric_name="ROI hat")
#plt7.set_figheight(10)
plt7.savefig(path + '/roi-hat.pdf')



plt8 = plot.plot_response_curves(media_mix_model=mmm, target_scaler=target_scaler, seed=SEED)
plt8.savefig(path + '/response_curves.pdf')

file_path = path + '/' + directory + ".pkl"
utils.save_model(media_mix_model=mmm, file_path=file_path)

contribution_df = plot.create_media_baseline_contribution_df(media_mix_model=mmm, target_scaler=target_scaler, channel_names = media_names)
contribution_df.to_excel(path + '/media_baseline_contribution.xlsx')

figs = [plt1, plt2, plt5, plt6, plt7, plt8]


In [ ]:
pdf = FPDF()
pdf.set_font('helvetica', size=14)

if not other_feature_names: other_feature_names = "None"
pdf.add_page()
pdf.multi_cell(w = 0, h = 5,
               txt = "File Name: " + file_name + "\n\n"
                   + "Model Name: " + model_name + "\n\n"
                   + "Media Channels: " + ', '.join([str(elem) for elem in media_names]) + "\n\n"
                   + "Extra Features: "  + other_feature_names + "\n"
               )
if(media_priors)
if not default_priors:
  pdf.write(txt="Custom Priors: \n")
  for x in parameterdict:
    pdf.write(txt= x + ': '+ str(parameterdict[x]) + "\n")
else:
  pdf.write(txt="Default Priors \n")

pdf.set_font('helvetica', size=14, style="B")
pdf.add_page()
canvas = FigureCanvas(plt1)
canvas.draw()
img = Image.fromarray(np.asarray(canvas.buffer_rgba()))

pdf.cell(w=0, h=5, txt="Model Fit on Training Data, " + str(split_point) + " weeks", align="C")
pdf.image(img, w=pdf.epw, x=10, y = 15)  # Make the image full width

canvas = FigureCanvas(plt2)
canvas.draw()
img = Image.fromarray(np.asarray(canvas.buffer_rgba()))
pdf.set_y(y=155)
pdf.cell(w=0, h=5, txt="Model Fit on Testing Data, " + str(test_size) + " weeks", align="C")
pdf.image(img, w=pdf.epw, x=10,y=160)  # Make the image full width

pdf.add_page()
canvas = FigureCanvas(plt10)
canvas.draw()
img = Image.fromarray(np.asarray(canvas.buffer_rgba()))
pdf.image(img, w=125, x=10,y=5)

canvas = FigureCanvas(plt6)
canvas.draw()
img = Image.fromarray(np.asarray(canvas.buffer_rgba()))
pdf.image(img, w=125, x=10,y=100)  # Make the image full width

#pdf.add_page()
canvas = FigureCanvas(plt7)
canvas.draw()
img = Image.fromarray(np.asarray(canvas.buffer_rgba()))
pdf.image(img, w=125, x=10,y=195)  # Make the image full width

pdf.add_page()
canvas = FigureCanvas(plt8)
canvas.draw()
img = Image.fromarray(np.asarray(canvas.buffer_rgba()))
pdf.image(img, w=pdf.epw, x=10,y=15)  # Make the image full width

pdf.output(path + model_name + "matplotlib.pdf")

In [ ]:
original_stdout = sys.stdout # Save a reference to the original standard output
text = '/content/drive/My Drive/LMMM-Data/' + directory + '/summary.txt'
with open(text ,'w') as f :
  sys.stdout = f # Change the standard output to the file we created.
  print("Model Name: " + model_name)
  print("Media Channels: " + ', '.join([str(elem) for elem in media_names]))
  print("Number of warmups: " + str(number_warmup))
  print("Number of samples: " + str(number_samples))
  print("Degrees of seasonality: " + str(degrees_seasonality))
  print("Custom priors: " + str(custom_priors))
  mmm.print_summary()
  sys.stdout = original_stdout